In [ ]:
#import libraries 
import numpy as np  
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
#import data 
red_wine = pd.read_csv("wine+quality\winequality-red.csv", sep= ';', delimiter=None) 
white_wine = pd.read_csv("wine+quality\winequality-white.csv", sep= ';', delimiter=None) 

#make a copy
rw = red_wine
ww = white_wine

Questions 
do we have to remove all spaces from column names for the ML modell to work? 

To-Do 
- remove duplicates 
- unify column names - remove spaces - example is in the movie data set book 
- check for null values 

- add column to each data set identifying red or white label - call it wine type 
    for loop? to fill the column 
- change data type to numerical?? 
- combine data sets 

Later 
- create new low medium high classification 
  low 1-4, medium 5-6, high 7-10 
- justification: our main focus is on what produces good quality wine. good is defined as above average. the average for red was 5.5 and the average for white was 5.9 
- so 5-6 is of medium quality because it represents the mean. anything below that is of low quality, and anything above that is considered high quality in this evalutation 
- also refereing to using the model for future data, 5 is the mediam of my quality range and extending that range upward will only increase the quality 


- maybe i could do a 5 Star Rating, make 5 groups, each with 2 points 

In [ ]:
#remove all duplicates 
rw.drop_duplicates(inplace=True)
ww.drop_duplicates(inplace=True)

In [ ]:
#unify column names 
rw.rename(columns = {'fixed acidity':'fixed_acidity','volatile acidity':'volatile_acidity','citric acid':'citric_acid','residual sugar':'residual_sugar','free sulfur dioxide':'free_sulfur_dioxide', 'total sulfur dioxide':'total_sulfur_dioxide'}, inplace=True)
ww.rename(columns = {'fixed acidity':'fixed_acidity','volatile acidity':'volatile_acidity','citric acid':'citric_acid','residual sugar':'residual_sugar','free sulfur dioxide':'free_sulfur_dioxide', 'total sulfur dioxide':'total_sulfur_dioxide'}, inplace=True)

In [ ]:
#add a column 
rw.insert(12, "colour", 'red')
ww.insert(12, "colour", 'white')
#could also do rw["colour"] = "red"

In [ ]:
#combine data sets 
data = [rw, ww] #its placing data ets on top of each other = ignor_index 
df_clean = pd.concat(data, ignore_index=True)
df_clean

In [ ]:
df_clean.shape

In [ ]:
#check colour column, data type 
df_clean["colour"].unique()

In [ ]:
#use One-Hot Encoder to convert the colour column from categorical str object to readable numbers 
#automatic assigning is 1 
df_clean["colour"] = (df_clean["colour"] == "red").astype(int)

In [ ]:
# .head looks weird so using value_counts to check count and dtype 
df_clean.colour.value_counts()

In [ ]:
mlm1 = df_clean.copy()
mlm1

Build the Model
- split the data 
- feature scaling 

In [ ]:
#split the data using the train_test_split function 

#X=mlm1.data
#y=mlm1.target

#y=x.colour
#x.drop(['colour'], axis=1)

#x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=39)

In [ ]:
#split data using numpy np.split
#to split by 3 instead of 2 
train, valid, test = np.split(mlm1.sample(frac=1), [int(0.6*len(mlm1)), int(0.8*len(mlm1))])
#.sample will shuffle my data 

In [ ]:
#fix the scale of all my numbers 
def scale_dataset(mlm1, oversample=False):
    X = mlm1[mlm1.columns[:-1]].values #calling all columns until the last one 
    y = mlm1[mlm1.columns[-1]].values #calling only the last column 

    scaler = StandardScaler() # check rules here for group splitting 
    X = scaler.fit_transform(X) 

    if oversample: 
        ros = RandomOverSampler()  
        X, y = ros.fit_resample(X, y)

    data = np.hstack((X, np.reshape(y, (-1, 1))))
    # y is only 2D, so call numpy reshape to make it  3Dimensional Item 
    # because y is only 1 column

    return data, X, y


In [ ]:
print(len(train[train["colour"] ==1 ])) # colour = red 
print(len(train[train["colour"] ==0 ])) # colour = white 
# you can import oversampling from imblearn to increase to lower sample to make the samples more even 

In [ ]:
print(len(valid[valid["colour"] ==1 ])) 
print(len(valid[valid["colour"] ==0 ]))

In [ ]:
#scaling data set using oversampling 
train, X_train, y_train = scale_dataset(train, oversample=True)
valid, X_valid, y_valid = scale_dataset(valid, oversample=False)
test, X_test, y_test = scale_dataset(test, oversample=False)

#Question: how do i know if the groups were evenly distributed before i "equalized" them. what if there were only 2 reds in the train one for ex. have those 2 just been multiplied by 100, i dont quite get it. 

In [ ]:
#red and white in train sample should now be the same number 
len(y_train)
sum(y_train == 1)

In [ ]:
sum(y_train == 0)
# and they are :) 

KNN - k-nearest neighbors 

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(X_train, y_train)

In [ ]:
y_pred = knn_model.predict(X_test)

In [ ]:
# y_pred should be the same y_test 
y_pred

In [ ]:
y_test

In [ ]:
#check classification report 
print(classification_report(y_test, y_pred)) 

In [ ]:
#accuracy on f1-score = 99% 

In [ ]:
# do a confusion matrix  
print(confusion_matrix(y_test, y_pred))
#google explaination